In [74]:
import requests
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from bs4 import BeautifulSoup
from openpyxl import Workbook
import openpyxl
import os
import json

In [14]:
import requests
from bs4 import BeautifulSoup
perfumes_urls = []
for i in tqdm(range(250)):
    url = f"https://www.olfastory.com/parfum.html?page={i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    product_titles = soup.find_all('div', {'class': 'c-product__title'})
    for title in product_titles:
        perfumes_urls.append(title.find("a")["href"])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [02:42<00:00,  1.53it/s]


In [17]:
perfumes_urls[0]

'https://www.olfastory.com/parfum/invictus-victory-elixir-de-paco-rabanne'

In [78]:
p_url = "https://www.olfastory.com/parfum/invictus-victory-elixir-de-paco-rabanne"

p_url = "https://www.olfastory.com/parfum/flower-kenzo-labsolue" 


parfums = {}
ingredients = {}
cpt = 0
for p_url in perfumes_urls:
    cpt = cpt + 1
    c_url = p_url + "/composition/#o"
    print(f"{cpt}/{len(perfumes_urls)} : {c_url}")
    response = requests.get(c_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        woman = soup.find_all('span', {'class': 'c-fonticon__icon--woman'})
        man = soup.find_all('span', {'class': 'c-fonticon__icon--man'})
        desc = soup.find_all('div', {'class':"o-table__cell--valignMiddle"})
        L = []
        for j in desc:
            if len(j.find_all('span', {'class': 'c-fonticon__icon--woman'})):
                L.append("woman")
            if len(j.find_all('span', {'class': 'c-fonticon__icon--man'})):
                    L.append("man")
            if len(j.find_all('span', {'class': 'c-fonticon__icon--mixed'})):
                L.append("mixed")
        
        pyr = soup.find_all("div", {'class':"c-pyramide__label"})
        uls = soup.find_all("ul", {'class':"c-list"})
        p_desc = {}
        p_desc["nom"] = p_url.split("/")[-1]
        p_desc["genre"] = L[0] if len(L) != 0 else "unknown"
        for p,u in zip(pyr, uls):
            type_n = (p.text.replace("Notes de ", "").capitalize())
            p_desc[type_n] = []
            for j in u:
                p_desc[type_n].append(j.find_all("a")[0]["href"].split("/")[-1])
                ingredients[j.find_all("a")[0]["href"].split("/")[-1]] = j.find_all("a")[0]["href"]
        parfums[p_url] = p_desc
        if(p_desc["genre"] == "unknown"):
            print(f"Unknown gender: {c_url}")
        parfums_json = json.dumps(parfums)
        ingredients_json = json.dumps(ingredients)
        parfums_f = open("parfums.json","w")
        ingredients_f = open("ingredients.json","w")
        parfums_f.write(parfums_json)
        ingredients_f.write(ingredients_json)
    except (Exception):
        print("Problem for URL:", p_url)

1/1986 : https://www.olfastory.com/parfum/invictus-victory-elixir-de-paco-rabanne/composition/#o
2/1986 : https://www.olfastory.com/parfum/la-petite-robe-noire-rose-cherry/composition/#o
3/1986 : https://www.olfastory.com/parfum/aqua-allegoria-harvest-rosa-rossa/composition/#o
4/1986 : https://www.olfastory.com/parfum/aqua-allegoria-harvest-nerolia-vetiver/composition/#o
5/1986 : https://www.olfastory.com/parfum/aqua-allegoria-harvest-madarine-basilic/composition/#o
6/1986 : https://www.olfastory.com/parfum/aqua-allegoria-forte-nerolia-vetiver/composition/#o
7/1986 : https://www.olfastory.com/parfum/good-girl-blush/composition/#o
8/1986 : https://www.olfastory.com/parfum/1-million-royal/composition/#o
9/1986 : https://www.olfastory.com/parfum/y-eau-de-parfum-intense/composition/#o
10/1986 : https://www.olfastory.com/parfum/eau-rochas-soleil-citron/composition/#o
11/1986 : https://www.olfastory.com/parfum/lady-million-royal/composition/#o
12/1986 : https://www.olfastory.com/parfum/her-f

In [82]:
len(parfums.keys())

1984

In [81]:
len(ingredients)

327